In [ ]:
import matplotlib.pyplot as plt
import os
os.environ.update(dict(CUDA_VISIBLE_DEVICES='3'))

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# data = torch.load('/ssd1/tta/imagenet_val_resnet50_lyrfts_full.pth')
data = torch.load('/ssd1/tta/imagenet_val_resnet50_shf_bn_full.pth')

In [ ]:
#features, labels
features_inc = data['features']
features_im = data['ifeatures'] # 50000, C (2048, 1024, 512, 256)
labels = data['labels']
logits_inc = data['logits']
logits_im = data['ilogits']
correct = data['correct'].type(torch.int)

layer = -1

def cidx(i):
    return labels == i

def cov(tensor: torch.Tensor, rowvar=True, bias=False):
    """Estimate a covariance matrix (np.cov)"""
    tensor = tensor if rowvar else tensor.transpose(-1, -2)
    tensor = tensor - tensor.mean(dim=-1, keepdim=True)
    factor = 1 / (tensor.shape[-1] - int(not bool(bias)))
    return factor * tensor @ tensor.transpose(-1, -2).conj()

def cls_distrib(c: int):
    ft: torch.Tensor = features_im[layer][cidx(c)].cuda() # 50, C
    cv = cov(ft, rowvar=False).type(torch.float64)
    L = torch.linalg.cholesky(cv + torch.eye(ft.shape[1]).cuda() * 1e-6)
    return ft.mean(0).cpu(), torch.cholesky_inverse(L).cpu() #cov(ft, rowvar=False) #ft.var(0).sqrt()

# cls_distribs = []
# for c in tqdm(range(1000)):
#     cls_distribs.append(cls_distrib(c))

In [ ]:
def get_distrib(data: torch.Tensor):
    data = data.type(torch.double)
    cov = data.T.cov()
    L, V = torch.linalg.eig(cov)
    L = L.type(torch.double)
    V = V.type(torch.double)
    L[L<0] = 0.
    ncov = V @ torch.diag(L) @ torch.linalg.inv(V)
    ninv = V @ torch.diag(L).inverse() @ torch.linalg.inv(V)
    mean = data.mean(0)
    print(mean.shape, ncov.shape)
    return mean, ncov, ninv, V, L

In [ ]:
savedata = []
for ft in features_im:
    mean, ncov, ninv, V, L = get_distrib(ft)
    savedata.append(dict(mean=mean, cov=ncov, cinv=ninv, V=V, L=L))

torch.save(savedata, '/ssd1/tta/imagenet_val_resnet50_distributions.pth')

In [ ]:
def mahalanobis(u, v, cov):
    delta = (u - v)
    m = torch.dot(delta, torch.matmul(cov.double(), delta))
    return torch.sqrt(m).float()

In [ ]:
mahalanobis(features_inc[2][3], savedata[2]['mean'], savedata[2]['cinv'])

In [ ]:
def mahalanobis(u, v, cov):
    delta = (u - v)
    m = torch.dot(delta, torch.matmul(cov.type(torch.float64).type(torch.float32), delta))
    print(f'{u.shape=} {v.shape=} {delta.shape=} {m.shape=} {cov.shape=}')
    print(m)
    return torch.sqrt(m)

def cls_dist(x: torch.Tensor, c: int): #x: 2048, c: int
    mean, cov = cls_distribs[c]
    # t = mahalanobis(x, mean.cuda(), cov.cuda())
    # print(t)
    t = ((x - mean.cuda()) ** 2).sum().sqrt()
    return t.cpu()
    # return ((x - cls_centers[c]) ** 2).sum().sqrt()

dists = []
# cls_dist(features_inc[layer][3].cuda(), labels[3])
# for c in tqdm(range(1000)):
#     x = features_inc[layer][cidx(c)].cuda() # 50, C
#     d = cls_dist(x, labels[c]).numpy()
#     # print(d.shape)
#     print(d)
#     break
#     dists.append(d)

for x, c in tqdm(zip(features_inc[layer], labels), total=len(labels)):
    dists.append(cls_dist(x.cuda(), labels[c]).numpy())
# dists = np.stack([cls_dist(x, labels[c]).numpy() for x, c in ])
dists = np.stack(dists)
# dists.shape

In [ ]:
cls_dist(features_inc[-1][1].cuda(), labels[1])

In [ ]:
cfd_src = logits_im.softmax(dim=1).max(dim=1)[0]
cfd_inc = logits_inc.softmax(dim=1).max(dim=1)[0]

cfd_dif = (cfd_inc - cfd_src)

# dists = np.stack([cls_dist(x, labels[c]).numpy() for x, c in zip(features_inc[layer], labels)])
# dists.shape
# ft_dif = nn.functional.cosine_similarity(features_inc[0], features_im[0])
# ft_dif = ((features_inc[0] - features_im[0]) ** 2).sqrt()
# ft_dif = ((features_inc[1] - features_im[1]) ** 2).sum(dim=1).sqrt()
# ft_dif.shape

In [ ]:
df = pd.DataFrame({
    'dists': dists,
    'cfd_inc': cfd_inc.numpy(),
    'cfd_dif': cfd_dif.numpy(),
    'correct': correct.numpy(),
})

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
df_dist = pd.Series(dists)
df_cfd = pd.Series(cfd_dif[cidx(10)].numpy())

In [ ]:
df_cfd.plot()

In [ ]:
df_dist.corr(df_cfd)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
scatter = ax.scatter(dists, cfd_inc, s=5, c=1-correct.numpy(), cmap='bwr')
ax.add_artist(ax.legend(scatter.legend_elements()[0], ['Correct', 'Wrong'], fontsize=20))
# ax.legend(['a', 'b'])
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
y=1-correct.numpy()
scatter = ax.scatter(dists, s=5, y=y, c=y, cmap='bwr')
ax.add_artist(ax.legend(scatter.legend_elements()[0], ['Correct', 'Wrong'], fontsize=20))
# ax.legend(['a', 'b'])
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.show()

In [ ]:
def corr(i):
    df_cfd = pd.Series(cfd_dif[cidx(i)].numpy().T)
    df_ft = pd.DataFrame(ft_dif[cidx(i)].numpy())  
    return df_ft.corrwith(df_cfd).item()

In [ ]:
corrs = pd.Series([corr(i) for i in range(1000)])

In [ ]:
corrs.describe()

In [ ]:
corrs.hist()

In [ ]:
from openTSNE import TSNE
import openTSNE.callbacks
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from tqdm import tqdm
class ProgressCallback(openTSNE.callbacks.Callback):
    def __init__(self, pbar: tqdm, step: int=1) -> None:
        super().__init__()
        self.pbar = pbar
        self.step = step

    def __call__(self, iteration, error, embedding):
        self.pbar.update(self.step)
        return False

In [ ]:
def visualize_tsne(features: np.ndarray, labels: np.ndarray, label_names: list[str]=None,
                   figsize=(10, 10), dimension=2, perplexity=30, legend_nrow=2):
    
    print(f'{features.shape=}, {labels.shape=}')

    with tqdm(total=750) as pbar:
        tsne = TSNE(n_jobs=8, 
                    n_components=dimension, 
                    perplexity=perplexity, 
                    callbacks_every_iters=1,
                    callbacks=ProgressCallback(pbar, 1))
        trained = tsne.fit(features)

    cluster = np.array(trained)

    print('t-SNE computed, waiting for plot...')

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot() if dimension < 3 else fig.add_subplot(projection='3d')
    
    classes = np.unique(labels)
    for i in classes:
        idx = np.where(labels == i)
        ax_args = dict(
            marker = '.' if i < 10 else 'o', 
            label = i if label_names is None else label_names[int(i)], 
            edgecolors = 'face' if i<10 else '#000000bb', 
            linewidths = 0.5
        )

        if dimension < 3:
            ax.scatter(cluster[idx, 0], cluster[idx, 1], **ax_args)
        else:
            ax.scatter(cluster[idx, 0], cluster[idx, 1] ,cluster[idx, 2], **ax_args)
            
    ax.autoscale()

    plt.legend(loc='lower center', ncol=len(classes)//legend_nrow, bbox_to_anchor=(0.5, -0.05))
    plt.axis('off')
    plt.show()

    return cluster, fig




In [ ]:
tsne_num = 2048
tsne_idx = np.random.choice(len(features_im[-1]), tsne_num)
tsne_fts = np.concatenate((features_im[-1][tsne_idx], features_inc[-1][tsne_idx]))
tsne_labels = np.concatenate((np.zeros(tsne_num), np.ones(tsne_num)))

visualize_tsne(tsne_fts, tsne_labels, ["ImageNet", "IN-C G5"])